In [3]:
import pandas as pd
import chardet
from mpl_toolkits.mplot3d import Axes3D  
from scipy.stats import norm 
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil

In [4]:
## function 1 takes as input a csv file and calculates the mean spectral-temporal signature 
# for each available label in the given column. 
# @param[in] inCsvfile : a csv file exported from PlotToSat or a merged csv file
# @param[in] col       : name of columns that medians will be calculated
# @param[in] outCsvfile: name of file to be exported
def getMeanSpectralTemporalSignatures(inCsvfile,col,outCsvfile):
    csvDF = pd.read_csv(inCsvfile,low_memory=False) 
    labels = list(csvDF.columns)
    if (col not in labels):
        raise Exception("ERROR: ", col, " not included in ", inCsvfile)
    newLabels = [col] 
    l = 0
    while (l<len(labels)):
        if ((labels[l][0].isdigit() and labels[l][1]=="_") or 
            (labels[l][0].isdigit() and labels[l][1].isdigit() and labels[l][2]=="_")):
            newLabels.append(labels[l])
        l=l+1
    newLabelsS1 = [col, '0_VHAsc','1_VHAsc','2_VHAsc','3_VHAsc', '4_VHAsc', '5_VHAsc','6_VHAsc', '7_VHAsc', '8_VHAsc', '9_VHAsc', '10_VHAsc','11_VHAsc', '0_VHDes','1_VHDes','2_VHDes','3_VHDes', '4_VHDes', '5_VHDes','6_VHDes', '7_VHDes', '8_VHDes', '9_VHDes', '10_VHDes','11_VHDes', '0_VVAsc','1_VVAsc','2_VVAsc','3_VVAsc', '4_VVAsc', '5_VVAsc','6_VVAsc', '7_VVAsc', '8_VVAsc', '9_VVAsc', '10_VVAsc','11_VVAsc', '0_VVDes','1_VVDes','2_VVDes','3_VVDes', '4_VVDes', '5_VVDes','6_VVDes', '7_VVDes', '8_VVDes', '9_VVDes', '10_VVDes','11_VVDes']
    newLabelsS2 = [col, '0_B1',  '0_B2', '0_B3', '0_B4', '0_B5', '0_B6', '0_B7', '0_B8', '0_B8A', '0_B9', '0_B11', '0_B12', '1_B1',  '1_B2', '1_B3', '1_B4', '1_B5', '1_B6', '1_B7', '1_B8', '1_B8A', '1_B9','1_B11', '1_B12', '2_B1',  '2_B2', '2_B3', '2_B4', '2_B5', '2_B6', '2_B7', '2_B8', '2_B8A', '2_B9', '2_B11', '2_B12', '3_B1', '3_B2', '3_B3', '3_B4', '3_B5', '3_B6', '3_B7', '3_B8', '3_B8A', '3_B9', '3_B11', '3_B12',  '4_B1', '4_B2', '4_B3', '4_B4', '4_B5', '4_B6', '4_B7', '4_B8', '4_B8A', '4_B9',  '4_B11', '4_B12', '5_B1',  '5_B2', '5_B3', '5_B4', '5_B5', '5_B6', '5_B7', '5_B8', '5_B8A', '5_B9', '5_B11', '5_B12', '6_B1', '6_B2', '6_B3', '6_B4', '6_B5', '6_B6', '6_B7', '6_B8', '6_B8A', '6_B9', '6_B11', '6_B12',  '7_B1',  '7_B2', '7_B3', '7_B4', '7_B5', '7_B6', '7_B7', '7_B8', '7_B8A', '7_B9', '7_B11', '7_B12', '8_B1','8_B2', '8_B3', '8_B4', '8_B5', '8_B6', '8_B7', '8_B8', '8_B8A', '8_B9',  '8_B11', '8_B12',  '9_B1', '9_B2', '9_B3', '9_B4', '9_B5', '9_B6', '9_B7', '9_B8', '9_B8A', '9_B9', '9_B11', '9_B12',  '10_B1', '10_B2', '10_B3', '10_B4', '10_B5', '10_B6', '10_B7', '10_B8', '10_B8A', '10_B9','10_B11', '10_B12',  '11_B1',  '11_B2', '11_B3', '11_B4', '11_B5', '11_B6', '11_B7', '11_B8', '11_B8A', '11_B9', '11_B11', '11_B12']
    print(newLabelsS1)
    print(newLabelsS2)
    
    csvDFS1 = csvDF.loc[:,newLabelsS1]
    csvDFS1=csvDFS1.groupby(col).mean()
    csvDFS1.to_csv(outCsvfile+"S1.csv")
    
    csvDFS2 = csvDF.loc[:,newLabelsS2]
    csvDFS2=csvDFS2.groupby(col).mean()
    csvDFS2.to_csv(outCsvfile+"S2.csv")
    
    print("File exported in ", outCsvfile+"S1.csv and ", outCsvfile+"S2.csv")  
    
# function 2 takes as input the output of function 1 and creates spectral-temporal signatures graphs for S2 and temporal signatures for S1   
def createSpectralTemporalGraphs(inCsvFile,outFilestart):
    inCsvFileS1 = inCsvFile+"S1.csv"
    csvDFS1 = pd.read_csv(inCsvFileS1)
    labels = list(csvDFS1.columns)
    for index, row in csvDFS1.iterrows():
        row1 = row
        tmplist = np.array(row1)
        rowlist = []
        for item in tmplist:
            rowlist = rowlist + [item]
        modulo = (len(rowlist)-1)%12
        if (modulo!=0 and len(rowlist<=0)):
            raise Exception("this is not a 12 months spectral temporal signature")      
        outImg = outFilestart+"_"+rowlist[0]+"_S1.jpg"
        data =  [rowlist[1:13],rowlist[13:25],rowlist[25:37],rowlist[37:49]]
        data = np.array(data)
        months = [0,1,2,3,4,5,6,7,8,9,10,11]
        plt.figure(figsize=(10, 9)) 
        plt.plot(months,data[0],label = "VV Ascending", color="#F5793A") # magenta
        plt.plot(months,data[1],label = "VV Descending", color="#A95AA1") # blue
        plt.plot(months,data[2],label = "VH Ascending", color="#85C0F9") # yellow
        plt.plot(months,data[3],label = "VH Descending", color="#0F2080") # green

        plt.xlabel('Month', fontsize=23)  # Adjust fontsize as needed
        plt.ylabel('Mean backscattering coefficient', fontsize=24) 
        plt.xticks(months, ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"], fontsize=16)  # Adjust fontsize as needed
        plt.xticks(fontsize = 21)
        plt.yticks(fontsize = 21)
        plt.ylim(-18, -8)
        plt.legend(fontsize=24)
        plt.savefig(outImg, dpi=100)
        plt.clf()
    
        
    inCSVFileS2 = inCsvFile+"S2.csv"   
    csvDFS2 = pd.read_csv(inCSVFileS2)
 
    labels = list(csvDFS2.columns)
    for index, row in csvDFS2.iterrows():
        row1 = row
        tmplist = np.array(row1)
        rowlist = []
        for item in tmplist:
            rowlist = rowlist + [item]
        modulo = (len(rowlist)-1)%12
        if (modulo!=0 and len(rowlist<=0)):
            raise Exception("this is not a 12 months spectral temporal signature")
        outImg = outFilestart+"_"+rowlist[0]+"_S2.jpg"
        print(outImg)
        data =  [rowlist[      1:12   +1], #'0_B1',  '0_B2', '0_B3', '0_B4', '0_B5', '0_B6', '0_B7', '0_B8', '0_B8A', '0_B9', '0_B11', '0_B12', 
                 rowlist[12   +1:12* 2+1], #'1_B1',  '1_B2', '1_B3', '1_B4', '1_B5', '1_B6', '1_B7', '1_B8', '1_B8A', '1_B9','1_B11', '1_B12', 
                 rowlist[12* 2+1:12* 3+1], #'2_B1',  '2_B2', '2_B3', '2_B4', '2_B5', '2_B6', '2_B7', '2_B8', '2_B8A', '2_B9', '2_B11', '2_B12', 
                 rowlist[12* 3+1:12* 4+1], #'3_B1', '3_B2', '3_B3', '3_B4', '3_B5', '3_B6', '3_B7', '3_B8', '3_B8A', '3_B9', '3_B11', '3_B12',  
                 rowlist[12* 4+1:12* 5+1], #'4_B1', '4_B2', '4_B3', '4_B4', '4_B5', '4_B6', '4_B7', '4_B8', '4_B8A', '4_B9',  '4_B11', '4_B12', 
                 rowlist[12* 5+1:12* 6+1], #'5_B1',  '5_B2', '5_B3', '5_B4', '5_B5', '5_B6', '5_B7', '5_B8', '5_B8A', '5_B9', '5_B11', '5_B12', 
                 rowlist[12* 6+1:12* 7+1], #'6_B1', '6_B2', '6_B3', '6_B4', '6_B5', '6_B6', '6_B7', '6_B8', '6_B8A', '6_B9', '6_B11', '6_B12',  
                 rowlist[12* 7+1:12* 8+1], #'7_B1',  '7_B2', '7_B3', '7_B4', '7_B5', '7_B6', '7_B7', '7_B8', '7_B8A', '7_B9', '7_B11', '7_B12', 
                 rowlist[12* 8+1:12* 9+1], #'8_B1','8_B2', '8_B3', '8_B4', '8_B5', '8_B6', '8_B7', '8_B8', '8_B8A', '8_B9',  '8_B11', '8_B12',  
                 rowlist[12* 9+1:12*10+1], #'9_B1', '9_B2', '9_B3', '9_B4', '9_B5', '9_B6', '9_B7', '9_B8', '9_B8A', '9_B9', '9_B11', '9_B12', 
                 rowlist[12*10+1:12*11+1], # '10_B1', '10_B2', '10_B3', '10_B4', '10_B5', '10_B6', '10_B7', '10_B8', '10_B8A', '10_B9','10_B11', 
                 rowlist[12*11+1:12*12+1]] #'10_B12',  '11_B1',  '11_B2', '11_B3', '11_B4', '11_B5', '11_B6', '11_B7', '11_B8', '11_B8A', '11_B9', '11_B11', '11_B12'
        data = np.array(data)
        data = np.transpose(data)     
        fig, (ax, bx) = plt.subplots(nrows=1, ncols=2, num=0, figsize=(16, 8),
                             subplot_kw={'projection': '3d'})
        for i in range(data.shape[1]):
            ax.plot3D(np.repeat(i, data.shape[0]), np.arange(data.shape[0]),data[:, i])
        gridX, gridY = np.mgrid[1:data.shape[0]:data.shape[0] * 1j,
                                1:data.shape[1]:data.shape[1] * 1j]
        ax.set_xlabel('Band')
        ax.set_ylabel('Month')
        ax.set_zlabel('Mean Spectral Value')
        bx.set_xlabel('Band')
        bx.set_ylabel('Month')
        bx.set_xticklabels(["B1","B3","B5","B7","B8A","B11", ""])
        ax.set_xticklabels(["B1","B1","B3","B5","B7","B8A","B11", ""])
        bx.set_yticklabels(["Jan",  "Mar",  "May","Jul",  "Sep",  "Nov"])
        ax.set_yticklabels(["Jan","Jan", "Mar", "May", "Jul", "Sep", "Nov"])
        ax.set_zlim(0, 3500)
        bx.set_zlim(0, 3500)
        pSurf = bx.plot_surface(gridY, gridX, data, cmap='viridis')
        pSurf = bx.plot_surface(gridY, gridX, data, cmap='viridis', vmin=0, vmax=3500)
        fig.colorbar(pSurf)
        plt.savefig(outImg)
        plt.clf()
    print("hello")


def createMultiTemporalGraphsFromOneFile(inCsvFile, outFilestart):
    inCsvFileS1 = inCsvFile
    csvDFS1 = pd.read_csv(inCsvFileS1)
    labels = list(csvDFS1.columns)

    # Create subplots for all classes
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

    class_names = ["VV Ascending", "VV Descending", "VH Ascending", "VH Descending"]

    for i in range(3):  # Assuming there are three classes
        row = csvDFS1.iloc[i]  # Get the row corresponding to the class
        rowlist = row.tolist()
        data = [rowlist[1:13], rowlist[13:25], rowlist[25:37], rowlist[37:49]]
        data = np.array(data)

        # Plot each class on its corresponding subplot
        for j in range(4):
            months = range(1, 13)
            axes[i].plot(months, data[j], label=class_names[j])

        axes[i].set_xlabel('Month')
        axes[i].set_ylabel('Mean backscattering coefficient')
        axes[i].set_xticks(months)
        axes[i].set_xticklabels(["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
        axes[i].legend()

    # Adjust layout
    plt.tight_layout()

    # Save or show the plot
    plt.savefig(outFilestart, dpi=100)
    plt.close(fig)
    plt.clf()
    print ("Image saved in ", outFilestart)    


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def createMultiTemporalGraphsFromMultipleFiles(inCsvFiles, outFilestart):
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))

    class_names = ["VV Ascending", "VV Descending", "VH Ascending", "VH Descending"]
    column_labels = ["Months in 2018", "Months in 2019", "Months in 2020"]
    y_labels = ["Broad-leaved deciduous\nMean backscattering coefficient", "Broad-leaved evergreen\nMean backscattering coefficient", "Needle-leaved evergreen\nMean backscattering coefficient"]

    for i in range(3):  # For each row (class)
        for j in range(3):  # For each column (file)
            inCsvFile = inCsvFiles[j]
            csvDFS1 = pd.read_csv(inCsvFile)
            row = csvDFS1.iloc[i]  # Get the row corresponding to the class
            rowlist = row.tolist()
            data = [rowlist[1:13], rowlist[13:25], rowlist[25:37], rowlist[37:49]]
            data = np.array(data)

            # Plot each class on its corresponding subplot
            for k in range(4):
                months = range(1, 13)
                axes[i, j].plot(months, data[k], label=class_names[k], linewidth=2)

            # Set axis labels font size
            if i == 2:
                axes[i, j].set_xlabel(column_labels[j], fontsize=16)
            if j == 0:
                axes[i, j].set_ylabel(y_labels[i], fontsize=16)

            # Set tick labels font size
            axes[i, j].tick_params(axis='both', which='major', labelsize=12)

            # Show ticks and labels only on the lower x-axis and the left y-axis
            if i == 2:
                axes[i, j].set_xticks(months)
                axes[i, j].set_xticklabels(["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
            else:
                axes[i, j].set_xticks([])

            if j == 0:
                axes[i, j].set_yticks(np.arange(-18.5, -8.5, 2))
            else:
                axes[i, j].set_yticks([])

            # Set legend in the middle of the graph
            if i == 2 and j == 2:
                axes[i, j].legend(loc='center', fontsize=14, bbox_to_anchor=(0.5, 0.5))

    # Set consistent y-axis limits across all subplots
    for i in range(3):
        for j in range(3):
            axes[i, j].set_ylim(-18.5, -8.5)

    # Adjust layout
    plt.tight_layout()

    # Save or show the plot
    plt.savefig(outFilestart, dpi=120)
    plt.close(fig)
    plt.clf()
    print ("Image saved in ", outFilestart)   




def createMultiSpectralTemporalGraphsLines(inCsvFiles, outFilestart):
    if len(inCsvFiles) != 3:
        raise ValueError("Input must contain 3 CSV files")

    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(16, 16), subplot_kw={'projection': '3d'})

    for idx, inCsvFile in enumerate(inCsvFiles):
        csvDFS = pd.read_csv(inCsvFile)
        labels = list(csvDFS.columns)

        for index, row in csvDFS.iterrows():
            row1 = row
            tmplist = np.array(row1)
            rowlist = []
            for item in tmplist:
                rowlist = rowlist + [item]
            modulo = (len(rowlist) - 1) % 12
            if modulo != 0 or len(rowlist) <= 0:
                raise Exception("This is not a 12 months spectral temporal signature")
            outImg = outFilestart + "_{}_S2.jpg".format(rowlist[0])
            print(outImg)
            data = [rowlist[(i * 12) + 1:((i + 1) * 12) + 1] for i in range(12)]  # Splitting into 12-month chunks
            data = np.array(data)
            data = np.transpose(data)
            for i in range(data.shape[1]):
                axes[index, idx].plot3D(np.repeat(i, data.shape[0]), np.arange(data.shape[0]), data[:, i])

            axes[index, idx].set_xlabel('Band')
            axes[index, idx].set_ylabel('Month')
            axes[index, idx].set_zlabel('Mean Spectral Value')
            axes[index, idx].set_xticklabels(["B1", "B3", "B5", "B7", "B8A", "B11", ""])
            axes[index, idx].set_yticklabels(["Jan", "Mar", "May", "Jul", "Sep", "Nov"])
            axes[index, idx].set_zlim(0, 3500)

    plt.tight_layout()
    plt.savefig(outFilestart)
    plt.clf()
    print("file saved in ", outFilestart)


def createMultiSpectralTemporalGraphsSurface(inCsvFiles, outFilestart):
    if len(inCsvFiles) != 3:
        raise ValueError("Input must contain 3 CSV files")

    fig = plt.figure(figsize=(18, 18))

    for idx, inCsvFile in enumerate(inCsvFiles):
        csvDFS = pd.read_csv(inCsvFile)
        labels = list(csvDFS.columns)

        for index, row in csvDFS.iterrows():
            row1 = row
            tmplist = np.array(row1)
            rowlist = []
            for item in tmplist:
                rowlist = rowlist + [item]
            modulo = (len(rowlist) - 1) % 12
            if modulo != 0 or len(rowlist) <= 0:
                raise Exception("This is not a 12 months spectral temporal signature")
            outImg = outFilestart + "_{}_S2.jpg".format(rowlist[0])
            print(outImg)
            data = [rowlist[(i * 12) + 1:((i + 1) * 12) + 1] for i in range(12)]  # Splitting into 12-month chunks
            data = np.array(data)
            X, Y = np.meshgrid(range(data.shape[0]), range(data.shape[1]))  # Transpose X and Y
            Z = data.astype(float)
            ax = fig.add_subplot(3, 4, index * 4 + idx + 1, projection='3d')
            surf = ax.plot_surface(Y, X, Z, cmap='viridis', edgecolor='none', vmin=0, vmax=3500)
            ax.set_xlabel('Band')
            ax.set_ylabel('Month')
            ax.set_zlabel('Mean Spectral Value')
            ax.set_yticklabels(["", "Jan", "Mar", "May", "Jul", "Sep", "Nov"])
            ax.set_xticklabels(["", "B1", "B3", "B5", "B7", "B8A", "B12"])  
            ax.set_zlim(0, 3500)
            ax.yaxis.set_rotate_label(True)  # Enable automatic rotation of the y-axis label
            ax.xaxis.set_rotate_label(True)  # Enable automatic rotation of the x-axis label


    plt.tight_layout()
    plt.savefig(outFilestart)
    plt.clf()
    print("file saved in ", outFilestart)


## function 2.51 takes as input the output of function 1
## takes as input 9 files and creates a 3x3 grid with multi-line graphs
# @parma[in] inCsvFiles: the list of the csv files to be imported
# @param[in] outFile: the name and directory of the image to be exported
def createS1TemporalMultiGraphs(row_labels, inCsvFiles, outFilestart):
    if len(row_labels) != 3 or len(inCsvFiles) != 3:
        raise ValueError("Input must contain 3 row labels and 3 CSV files")

    fig, axs = plt.subplots(3, 3, figsize=(18, 18))

    for i, (row_label, inCsvFile) in enumerate(zip(row_labels, inCsvFiles)):
        inCsvFileS1 = inCsvFile 
        csvDFS1 = pd.read_csv(inCsvFileS1)
        labels = list(csvDFS1.columns)

        for label in row_label:
            for index, row in csvDFS1.iterrows():
                if row[0] == label:
                    print ( row[0])
                    row1 = row
                    tmplist = np.array(row1)
                    rowlist = []
                    for item in tmplist:
                        rowlist = rowlist + [item]
                    modulo = (len(rowlist) - 1) % 12
                    if (modulo != 0 and len(rowlist <= 0)):
                        raise Exception("this is not a 12 months spectral temporal signature")
                    data = [rowlist[25:36], rowlist[37:48], rowlist[1:12], rowlist[13:24]]
                    data = np.array(data)
                    months = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

                    ax = axs[i // 3, i % 3]
                    ax.plot(months, data[0], label="VV Ascending", color="#D81B60")  # magenta
                    ax.plot(months, data[1], label="VV Descending", color="#1E88E5")  # blue
                    ax.plot(months, data[2], label="VH Ascending", color="#FFC107")  # yellow
                    ax.plot(months, data[3], label="VH Descending", color="#004D40")  # green
                    ax.set_xlabel('Month', fontsize=12)
                    ax.set_ylabel('Mean backscattering coefficient', fontsize=12)
                    ax.set_xticks(months)
                    ax.set_xticklabels(
                        ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
                        fontsize=10)
                    ax.set_yticks(fontsize=10)
                    ax.set_ylim(-18, -9)
                    ax.legend(fontsize=12)
                    ax.set_title(inCsvFile, fontsize=14)

    plt.tight_layout()
    plt.savefig(outFilestart + "_spectral_temporal_graphs.jpg", dpi=100)
    plt.show()

# function 3 takes as input the outpus of function 1 and produces the annual mean for each band 
def getMeanPerBandfromMeans(inCsvFile,col,outCsvfile):
    inCsvFileS1 = inCsvFile+"S1.csv"
    inCsvFileS2 = inCsvFile+"S2.csv"

    csvDFS2 = pd.read_csv(inCsvFileS2)
    # '0_B1',  '0_B2', '0_B3', '0_B4', '0_B5', '0_B6', '0_B7', '0_B8', '0_B8A', '0_B9', '0_B11', '0_B12', 
    dfS2  = csvDFS2[[col]]
    dfS2 ['B1' ] = csvDFS2 [['0_B1', '1_B1', '2_B1', '3_B1', '4_B1', '5_B1', '6_B1', '7_B1', '8_B1', '9_B1', '10_B1', '11_B1' ]].mean(axis=1) # '0_B1', 
    dfS2 ['B2' ] = csvDFS2 [['0_B2', '1_B2', '2_B2', '3_B2', '4_B2', '5_B2', '6_B2', '7_B2', '8_B2', '9_B2', '10_B2', '11_B2' ]].mean(axis=1) # '0_B2', 
    dfS2 ['B3' ] = csvDFS2 [['0_B3', '1_B3', '2_B3', '3_B3', '4_B3', '5_B3', '6_B3', '7_B3', '8_B3', '9_B3', '10_B3', '11_B3' ]].mean(axis=1) # '0_B3
    dfS2 ['B4' ] = csvDFS2 [['0_B4', '1_B4', '2_B4', '3_B4', '4_B4', '5_B4', '6_B4', '7_B4', '8_B4', '9_B4', '10_B4', '11_B4' ]].mean(axis=1) # '0_B4', 
    dfS2 ['B5' ] = csvDFS2 [['0_B5', '1_B5', '2_B5', '3_B5', '4_B5', '5_B5', '6_B5', '7_B5', '8_B5', '9_B5', '10_B5', '11_B5' ]].mean(axis=1) # '0_B5',
    dfS2 ['B6' ] = csvDFS2 [['0_B6', '1_B6', '2_B6', '3_B6', '4_B6', '5_B6', '6_B6', '7_B6', '8_B6', '9_B6', '10_B6', '11_B6' ]].mean(axis=1) # '0_B6',
    dfS2 ['B7' ] = csvDFS2 [['0_B7', '1_B7', '2_B7', '3_B7', '4_B7', '5_B7', '6_B7', '7_B7', '8_B7', '9_B7', '10_B7', '11_B7' ]].mean(axis=1) # '0_B7',
    dfS2 ['B8' ] = csvDFS2 [['0_B8', '1_B8', '2_B8', '3_B8', '4_B8', '5_B8', '6_B8', '7_B8', '8_B8', '9_B8', '10_B8', '11_B8' ]].mean(axis=1) # '0_B8',
    dfS2 ['B8A'] = csvDFS2 [['0_B8A','1_B8A','2_B8A','3_B8A','4_B8A','5_B8A','6_B8A','7_B8A','8_B8A','9_B8A','10_B8A','11_B8A']].mean(axis=1) # '0_B8A',
    dfS2 ['B9' ] = csvDFS2 [['0_B9', '1_B9', '2_B9', '3_B9', '4_B9', '5_B9', '6_B9', '7_B9', '8_B9', '9_B9', '10_B9', '11_B9' ]].mean(axis=1) # '0_B9',
    dfS2 ['B11'] = csvDFS2 [['0_B11','1_B11','2_B11','3_B11','4_B11','5_B11','6_B11','7_B11','8_B11','9_B11','10_B11','11_B11']].mean(axis=1) # '0_B11',
    dfS2 ['B12'] = csvDFS2 [['0_B12','1_B12','2_B12','3_B12','4_B12','5_B12','6_B12','7_B12','8_B12','9_B12','10_B12','11_B12']].mean(axis=1) # '0_B12',
    dfS2.to_csv(outCsvfile+"S2.csv")
    print ("Means saved in ", outCsvfile+"S2.csv")

    csvDFS1 = pd.read_csv(inCsvFileS1)
    # '0_VHAsc' 0_VHDes 0_VVAsc 0_VVDes
    dfS1 =  csvDFS1[[col]]
    dfS1['VHAsc'] = csvDFS1[['0_VHAsc', '1_VHAsc', '2_VHAsc', '3_VHAsc', '4_VHAsc', '5_VHAsc', '6_VHAsc', '7_VHAsc', '8_VHAsc', '9_VHAsc', '10_VHAsc', '11_VHAsc']].mean(axis=1) # '0_VHAsc',
    dfS1['VVAsc'] = csvDFS1[['0_VVAsc', '1_VVAsc', '2_VVAsc', '3_VVAsc', '4_VVAsc', '5_VVAsc', '6_VVAsc', '7_VVAsc', '8_VVAsc', '9_VVAsc', '10_VVAsc', '11_VVAsc']].mean(axis=1)  # '0_VVAsc',  
    dfS1['VHDes'] = csvDFS1[['0_VHDes', '1_VHDes', '2_VHDes', '3_VHDes', '4_VHDes', '5_VHDes', '6_VHDes', '7_VHDes', '8_VHDes', '9_VHDes', '10_VHDes', '11_VHDes']].mean(axis=1)  # '0_VHDes',      
    dfS1['VVDes'] = csvDFS1[['0_VVDes', '1_VVDes', '2_VVDes', '3_VVDes', '4_VVDes', '5_VVDes', '6_VVDes', '7_VVDes', '8_VVDes', '9_VVDes', '10_VVDes', '11_VVDes']].mean(axis=1)  # '0_VVDes',  
    dfS1.to_csv(outCsvfile+"S1.csv")
    print ("Means saved in ", outCsvfile+"S1.csv")

# function 4 takes as input the outpus of PlotToSat and produces the annual mean for each band 
def getMeanPerBand(inCsvFile,col,outCsvfile):
    csvDFS = pd.read_csv(inCsvFile)
    # '0_B1',  '0_B2', '0_B3', '0_B4', '0_B5', '0_B6', '0_B7', '0_B8', '0_B8A', '0_B9', '0_B11', '0_B12', 
    dfS2  = csvDFS[[col]]
    dfS2 ['B1' ] = csvDFS [['0_B1', '1_B1', '2_B1', '3_B1', '4_B1', '5_B1', '6_B1', '7_B1', '8_B1', '9_B1', '10_B1', '11_B1' ]].mean(axis=1) # '0_B1', 
    dfS2 ['B2' ] = csvDFS [['0_B2', '1_B2', '2_B2', '3_B2', '4_B2', '5_B2', '6_B2', '7_B2', '8_B2', '9_B2', '10_B2', '11_B2' ]].mean(axis=1) # '0_B2', 
    dfS2 ['B3' ] = csvDFS [['0_B3', '1_B3', '2_B3', '3_B3', '4_B3', '5_B3', '6_B3', '7_B3', '8_B3', '9_B3', '10_B3', '11_B3' ]].mean(axis=1) # '0_B3
    dfS2 ['B4' ] = csvDFS [['0_B4', '1_B4', '2_B4', '3_B4', '4_B4', '5_B4', '6_B4', '7_B4', '8_B4', '9_B4', '10_B4', '11_B4' ]].mean(axis=1) # '0_B4', 
    dfS2 ['B5' ] = csvDFS [['0_B5', '1_B5', '2_B5', '3_B5', '4_B5', '5_B5', '6_B5', '7_B5', '8_B5', '9_B5', '10_B5', '11_B5' ]].mean(axis=1) # '0_B5',
    dfS2 ['B6' ] = csvDFS [['0_B6', '1_B6', '2_B6', '3_B6', '4_B6', '5_B6', '6_B6', '7_B6', '8_B6', '9_B6', '10_B6', '11_B6' ]].mean(axis=1) # '0_B6',
    dfS2 ['B7' ] = csvDFS [['0_B7', '1_B7', '2_B7', '3_B7', '4_B7', '5_B7', '6_B7', '7_B7', '8_B7', '9_B7', '10_B7', '11_B7' ]].mean(axis=1) # '0_B7',
    dfS2 ['B8' ] = csvDFS [['0_B8', '1_B8', '2_B8', '3_B8', '4_B8', '5_B8', '6_B8', '7_B8', '8_B8', '9_B8', '10_B8', '11_B8' ]].mean(axis=1) # '0_B8',
    dfS2 ['B8A'] = csvDFS [['0_B8A','1_B8A','2_B8A','3_B8A','4_B8A','5_B8A','6_B8A','7_B8A','8_B8A','9_B8A','10_B8A','11_B8A']].mean(axis=1) # '0_B8A',
    dfS2 ['B9' ] = csvDFS [['0_B9', '1_B9', '2_B9', '3_B9', '4_B9', '5_B9', '6_B9', '7_B9', '8_B9', '9_B9', '10_B9', '11_B9' ]].mean(axis=1) # '0_B9',
    dfS2 ['B11'] = csvDFS [['0_B11','1_B11','2_B11','3_B11','4_B11','5_B11','6_B11','7_B11','8_B11','9_B11','10_B11','11_B11']].mean(axis=1) # '0_B11',
    dfS2 ['B12'] = csvDFS [['0_B12','1_B12','2_B12','3_B12','4_B12','5_B12','6_B12','7_B12','8_B12','9_B12','10_B12','11_B12']].mean(axis=1) # '0_B12',
    dfS2.to_csv(outCsvfile+"S2.csv")
    print ("Means saved in ", outCsvfile+"S2.csv")

    # '0_VHAsc' 0_VHDes 0_VVAsc 0_VVDes
    dfS1 =  csvDFS[[col]]
    dfS1['VHAsc'] = csvDFS[['0_VHAsc', '1_VHAsc', '2_VHAsc', '3_VHAsc', '4_VHAsc', '5_VHAsc', '6_VHAsc', '7_VHAsc', '8_VHAsc', '9_VHAsc', '10_VHAsc', '11_VHAsc']].mean(axis=1) # '0_VHAsc',
    dfS1['VVAsc'] = csvDFS[['0_VVAsc', '1_VVAsc', '2_VVAsc', '3_VVAsc', '4_VVAsc', '5_VVAsc', '6_VVAsc', '7_VVAsc', '8_VVAsc', '9_VVAsc', '10_VVAsc', '11_VVAsc']].mean(axis=1)  # '0_VVAsc',  
    dfS1['VHDes'] = csvDFS[['0_VHDes', '1_VHDes', '2_VHDes', '3_VHDes', '4_VHDes', '5_VHDes', '6_VHDes', '7_VHDes', '8_VHDes', '9_VHDes', '10_VHDes', '11_VHDes']].mean(axis=1)  # '0_VHDes',      
    dfS1['VVDes'] = csvDFS[['0_VVDes', '1_VVDes', '2_VVDes', '3_VVDes', '4_VVDes', '5_VVDes', '6_VVDes', '7_VVDes', '8_VVDes', '9_VVDes', '10_VVDes', '11_VVDes']].mean(axis=1)  # '0_VVDes',  
    dfS1.to_csv(outCsvfile+"S1.csv")
    print ("Means saved in ", outCsvfile+"S1.csv")
    
    
# function 5 that adds NDVI to the S2 output of function 3 or 4 
def addNDVI(inCsvFile, outCsvFile):
    dfS2 = pd.read_csv(inCsvFile+"S2.csv")
    dfS2['NDVI'] = (dfS2['B8']-dfS2['B4'])/(dfS2['B8']+dfS2['B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    dfS2.to_csv(outCsvFile)
    print (outCsvFile, "saved")
    


    
# function 6 that takes as input a csv file and exports a histogram with the distribution line
# @param[in] col, the name of the colmun of which the histogram will be derived
def saveHist(inCsvFile, col, outHist):
    df = pd.read_csv(inCsvFile) 
    data = np.array(df[col].tolist())  
    data = [i for i in data if i is not None]
    data = [x for x in data if ~np.isnan(x)]
    data = [i for i in data if i!='nan']
    mu, std = norm.fit(data)  
  
    # Plot the histogram. 
    plt.hist(data, bins=25, density=True, alpha=0.6, color='b') 
    
    # Plot the PDF. 
    xmin, xmax = plt.xlim() 
    x = np.linspace(xmin, xmax, 100) 
    p = norm.pdf(x, mu, std) 
    
    plt.plot(x, p, 'k', linewidth=2) 
    title = "Mean: {:.2f}, Std {:.2f}".format(mu, std) 
    plt.title(title)   
    plt.savefig(outHist)
    plt.clf()


def saveMultiHist(inCsvFiles, col_name, outHist, row_labels=None, col_labels=None):
    fig, axs = plt.subplots(3, 3, figsize=(18, 15), sharex=True, sharey=True)
    
    for i, csv_file in enumerate(inCsvFiles):
        row = i // 3
        col = i % 3
        
        df = pd.read_csv(csv_file)
        data = np.array(df[col_name].dropna())
        mu, std = norm.fit(data)  
            
        ax = axs[row, col]
        counts, bins, _ = ax.hist(data, bins=25, density=False, alpha=0.6, color='b')  # Density=False to plot counts

        mode_value = bins[np.argmax(counts)]  # Finding the mode value
        
        xmin, xmax = 0, 1.25  # Limiting x-axis from 0 to 1.25
        ax.set_xlim(xmin, xmax)
        
        x = np.linspace(xmin, xmax, 100) 
        p = norm.pdf(x, mu, std) 

        ax.plot(x, p * len(data) * np.diff(bins)[0], 'k', linewidth=2)  # Plotting PDF scaled by total count
        
        title = "Mean: {:.2f}, Std {:.2f}, Mode {:.2f}".format(mu, std, mode_value)  # Adding mode to the title
        ax.set_title(title, fontsize=14)
        
    # Add row labels
    if row_labels:
        for i, label in enumerate(row_labels):
            axs[i, 0].set_ylabel(label, fontsize=14)
    
    # Add column labels
    if col_labels:
        for j, label in enumerate(col_labels):
            axs[2, j].set_xlabel(label, fontsize=14)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjusting layout slightly to ensure labels are visible
    plt.savefig(outHist)
    plt.close(fig)

## function 7 that takes as input a csv file and exports a csv files for each available label in a given column
# for each available label in the given column. 
# @param[in] inCsvfile : a csv file exported from PlotToSat or a merged csv file
# @param[in] col       : name of columns that medians will be calculated
# @param[in] outCsvfile: name of file to be exported
def divideDataAccordingLabels(inCsvfile,col,outCsvfolder):
    if (os.path.exists(outCsvfolder)):
       print("WARNING: Directory ", outCsvfolder, "exist.")
    else:
        os.mkdir(outCsvfolder)
    
    csvDF = pd.read_csv(inCsvfile,low_memory=False) 
    
    dfs = [x.reset_index(drop=True) for _, x in csvDF.groupby(col)]
    [x.to_csv(f"{outCsvfolder}{x[col][0]}.csv", index=False) for x in dfs]    
    
 
## function 8 for each biome get th means and the hist graphs using previous graphs and spectral-temporal graphs
def getAll(inCsvFile,col,OutFolder):
    if (OutFolder[-1]!=["/"]):
        OutFolder = OutFolder+"/"
    
    if (os.path.exists(OutFolder)):
       print("WARNING: Directory ", OutFolder, "exist. ", OutFolder, "is removed!")
       shutil.rmtree(OutFolder)
    os.mkdir(OutFolder)
       
    getMeanSpectralTemporalSignatures(inCsvFile,col,OutFolder+"classMeans.csv")
    createSpectralTemporalGraphs(OutFolder+"classMeans.csv",OutFolder+"classMeans") 
    
    getMeanPerBand(inCsvFile,col,OutFolder+"_annualMeans.csv")
    addNDVI(OutFolder+"_annualMeans.csv", OutFolder+"_annualMeans_NDVI.csv")
    divideDataAccordingLabels(OutFolder+"_annualMeans_NDVI.csv",col,OutFolder)

    csvDFmeans = pd.read_csv(OutFolder+"classMeans.csvS1.csv")
    labels = np.array(csvDFmeans[col].tolist()) # the unique labels of the column of interest
    
    labels = [i for i in labels if i is not None]
    labels = [i for i in labels if i!='nan']
    print (labels)
    
    for label in labels :
        saveHist(OutFolder+label+".csv","NDVI",OutFolder+label+"_hist.jpg") 
         
## function 9 takes the output of PlotToSat and adds the NDVI of each month
def addNDVIpermonth(inCsvFile,Col,PlotsIdCol,outCsvFile):
    df = pd.read_csv(inCsvFile,low_memory=False) 
    df[ '0_NDVI'] = (df[ '0_B8']-df[ '0_B4'])/(df[ '0_B8']+df[ '0_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df[ '1_NDVI'] = (df[ '1_B8']-df[ '1_B4'])/(df[ '1_B8']+df[ '1_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df[ '2_NDVI'] = (df[ '2_B8']-df[ '2_B4'])/(df[ '2_B8']+df[ '2_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df[ '3_NDVI'] = (df[ '3_B8']-df[ '3_B4'])/(df[ '3_B8']+df[ '3_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df[ '4_NDVI'] = (df[ '4_B8']-df[ '4_B4'])/(df[ '4_B8']+df[ '4_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df[ '5_NDVI'] = (df[ '5_B8']-df[ '5_B4'])/(df[ '5_B8']+df[ '5_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df[ '6_NDVI'] = (df[ '6_B8']-df[ '6_B4'])/(df[ '6_B8']+df[ '6_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df[ '7_NDVI'] = (df[ '7_B8']-df[ '7_B4'])/(df[ '7_B8']+df[ '7_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df[ '8_NDVI'] = (df[ '8_B8']-df[ '8_B4'])/(df[ '8_B8']+df[ '8_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df[ '9_NDVI'] = (df[ '9_B8']-df[ '9_B4'])/(df[ '9_B8']+df[ '9_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df['10_NDVI'] = (df['10_B8']-df['10_B4'])/(df['10_B8']+df['10_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df['11_NDVI'] = (df['11_B8']-df['11_B4'])/(df['11_B8']+df['11_B4'])   # NDVI = (NIR-B8 - RED-B4) / (NIR-B8 + RED-B4) 
    df = df[[Col, PlotsIdCol, "0_NDVI", "1_NDVI","2_NDVI","3_NDVI", "4_NDVI", "5_NDVI", "6_NDVI", "7_NDVI", "8_NDVI", "9_NDVI", "10_NDVI", "11_NDVI"]]
    df.to_csv(outCsvFile)
    print ("Csv file with NDVs export in ", outCsvFile)


## function 9.1 takes as input the outputs of function 9 or any other csv files with the same structure and average them correspondingly. 
## e.g., an example is to take the monthly NDVI for 2018, 2019 and 2020 and it will return the average monthly NDVI of the three given years
## it further removes empty lines
## @param [in] thress the threshold for empty lines, e.g., if thres = 8 if 8 or more values are empty within a row then the row is removed
def aveCsvFiles(listOfFiles, col, PlotsIdCol, outfile, thres):
    if (listOfFiles == []):
        raise Exception("list is empty")
    df1 =  pd.read_csv(listOfFiles[0],low_memory=False) 
    df1.sort_values(by=PlotsIdCol,ascending=True)

    if(len(listOfFiles)==1):
        df1.to_csv(outfile)
        print("   *** EXIT SUCCESS   ***")
        return
       
    
    df2 =  pd.read_csv(listOfFiles[1],low_memory=False) 
    df2.sort_values(by=PlotsIdCol,ascending=True)
    count = 1 
    for col_name in df1.columns: 
        if(count==1 or count==2):
            count = count+1
            continue
        df1[col_name] = pd.to_numeric(df1[col_name], errors='coerce')
        df2[col_name] = pd.to_numeric(df2[col_name], errors='coerce')
        if(col_name==col or col_name==PlotsIdCol):
           continue
        for idx, value in enumerate(df2[col_name]):
            val1 = df1.at[idx, col_name]
            val2 = df2.at[idx, col_name] 
            if pd.isna(val2) or val2 is None:
                val2 = val1
            if pd.isna(val1) or val1 is None:
                val1 = val2
            df1.at[idx, col_name] = (val1+val2)/2

        
    count = 0
    for l in listOfFiles :
        if(count==1 or count==2):
            count = count+1
            continue
        
        df2 = pd.read_csv(l)
        df2[col] = pd.to_numeric(df2[col], errors='coerce')
        for idx, value in enumerate(df2[col_name]):
            val1 = df1.at[idx, col_name]
            val2 = df2.at[idx, col_name] 
            if pd.isna(val2) or val2 is None:
                val2 = val1
            if pd.isna(val1) or val1 is None:
                val1 = val2
            df1.at[idx, col_name] = (val1+val2)/2
   


    for index, row in df1.iterrows():
        count = 0
        nullVals = 0 
        for column in df1:
            if(count==1 or count==2):
                count = count+1
                continue
            val1 = df1.at[index, column]
            if pd.isna(val1) or val1 is None:
                nullVals=nullVals+1
        if(nullVals>=thres):
           df1=df1.drop(index)

    df1.to_csv(outfile)
    print ("***   EXIT SUCCESS   ***")
    
    
## function 10 takes as input the output of function 9, for each unique label in Col it gets the mean for each monthly NDVI 
def getMeanNDVIsPerMonthForClass(inCsvFile,Col,PlotsIdCol, outCsvFile):
    csvDF = pd.read_csv(inCsvFile,low_memory=False)
    # '0_NDVI',  '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI',  '9_NDVI', '10_NDVI', '11_NDVI', 
    labels = list(csvDF.columns)
    if (Col not in labels):
        raise Exception("ERROR: ", Col, " not included in ", inCsvFile)
    newLabels = [Col] 
    l = 0
    while (l<len(labels)):
        if ((labels[l][0].isdigit() and labels[l][1]=="_") or 
            (labels[l][0].isdigit() and labels[l][1].isdigit() and labels[l][2]=="_")):
            newLabels.append(labels[l])
        l=l+1
    
    newLabelsS2 = [PlotsIdCol, Col, '0_NDVI',  '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI',  '9_NDVI', '10_NDVI', '11_NDVI']
    print(newLabelsS2)    
    csvDFS2 = csvDF.loc[:,newLabelsS2]
    csvDFS2.to_csv(outCsvFile+"_allNDVIdata.csv")
    csvDFS2=csvDFS2.groupby(Col).mean()
    csvDFS2.to_csv(outCsvFile)
    print("Mean monthy NDVIs of class ", Col, " stored in ", outCsvFile)


    
## function 10.5 takes as input the outputs of function 10 and finds the best month to distingished clasess of interest
# inCsvFile+_allNDvidata.csv was exported in function 10.5 and used here
def getMonthForBetterDistingishingClasses(inCsvFile,Col):
    dfMean = pd.read_csv(inCsvFile, low_memory=False)
    dfMean = dfMean.drop(dfMean[dfMean[Col] == 'mixed'].index)
    dfMean['count'] = [0,0,0,0]
    sqrDifs = np.zeros(12)
    dfAll  = pd.read_csv(inCsvFile+"_allNDVIdata.csv", low_memory=False)
    dfAll.set_index(Col, inplace=True)
    #dfAll = dfAll.grouby(Col)
    dfAll = dfAll.drop(dfMean[dfMean[Col] == 'mixed'].index)
    print(dfMean)
    print("sqrDifs = ",sqrDifs)
    
    
        
## function 11 takes as input the output of function 10 and creates a multiline graph for the mean NDVIs of each class
def createGraphOfNDVIs(inCsvFile, Col, outImg):
    df = pd.read_csv(inCsvFile, low_memory=False)
    df.set_index(Col, inplace=True)
    plt.figure(figsize=(14,6),  dpi=300)
    df.T.plot(marker='o', linestyle='-')
    plt.title("NDVI during 2019 of different forest types")
    plt.xlabel("Months")
    plt.ylabel("NDVI Values")
    plt.legend(title="Forest Types", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.grid(True)
    plt.savefig(outImg)
    plt.show()
    plt.clf()
    print("Image saved in ", outImg)
    
    
## function 12 takes as input the output of function 9 and creates a multiline graph for the mean and std NDVIs of each class 
# # NOT WORKING
def creatGraphOfNDVIsWithStd(inCsvFile,Col,outImg):
    df = pd.read_csv(inCsvFile, low_memory=False)

    # Set the 'Col' column as the index
    df.set_index(Col, inplace=True)
    

    # Extract mean and std values
    means = df.groupby(Col).mean()
    stds = df.groupby(Col).std().fillna(df.groupby(Col).last())
    
    print (means)
    print (stds)
    print ("HELLO WORLDS")

    # Plot the multiline graph with mean and std
    plt.figure(figsize=(14, 6), dpi=300)
    for label, mean_values, std_values in zip(means.index, means.values.T, stds.values.T):
        plt.errorbar(x=means.columns, y=mean_values, yerr=std_values, marker='o', linestyle='-', label=label, alpha=0.7)

    # Customize the plot
    plt.title("Mean NDVI during 2019 of different forest types with Standard Deviation")
    plt.xlabel("Months")
    plt.ylabel("NDVI Values")
    plt.legend(title="Forest Types", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.grid(True)
    
    # Save the plot to the specified output image file
    plt.savefig(outImg)
    
    # Show the plot
    plt.show()

    # Clear the plot
    plt.clf()

    print("Image saved in ", outImg)


## function 13 takes as input a txt file and returns stats about execution time - file manually created by copying GEE output in Tasks
# locations of files hardcoded because run only once
def getExecutionTimeStatsforFile(inFile):
  
    file = open(inFile, 'r')
    lines = file.readlines()
    count = 0
    allmins = []
    S2_stdD_mins = []
    S2_mean_mins = []
    S1_stdD_mins = []
    S1_mean_mins = []
    invalidValues=0
    for line in lines: 
        if count%2 != 0 : 
            line  = line.replace('m', '')
            line  = line.replace('\n', '')
            try: 
                mins = int(line)
                allmins = allmins + [mins]
            except:
                print ("WARNING: one invalid value")
                invalidValues = invalidValues+1
                allmins = allmins + [-1]
        count = count + 1

    allmins=np.array(allmins)
    for x in range (0,len(allmins)):
        if( x%4 == 0):
            S2_stdD_mins = S2_stdD_mins + [allmins[x]]
        if (x%4 == 1):
            S2_mean_mins = S2_mean_mins + [allmins[x]]
        if (x%4 == 2):
            S1_stdD_mins = S1_stdD_mins + [allmins[x]]
        if (x%4 == 3):
            S1_mean_mins = S1_mean_mins + [allmins[x]]
            
    print( "NAME, MEAN, STD, LEN, INVALID")
    
    S2_stdD_minsInv = len(S2_stdD_mins)
    S2_stdD_mins = [ v for v in S2_stdD_mins if v  >= 0 ]
    S2_stdD_mins = np.array(S2_stdD_mins)
    S2_stdD_minsInv = S2_stdD_minsInv-len(S2_stdD_mins)
    print ("S2_stdD_mins", round(S2_stdD_mins.mean(),3), round(S2_stdD_mins.std(),3),len(S2_stdD_mins),S2_stdD_minsInv)
    
    S2_mean_minsInv = len(S2_mean_mins)
    S2_mean_mins = [ v for v in S2_mean_mins if v  >= 0]
    S2_mean_mins = np.array(S2_mean_mins)
    S2_mean_minsInv = S2_mean_minsInv-len(S2_mean_mins)
    print ("S2_mean_mins", round(S2_mean_mins.mean(),3), round(S2_mean_mins.std(),3),len(S2_mean_mins),S2_mean_minsInv)
    
    S1_stdD_minsInv = len(S1_stdD_mins)
    S1_stdD_mins = [ v for v in S1_stdD_mins if v  >= 0]
    S1_stdD_mins = np.array(S1_stdD_mins)
    S1_stdD_minsInv = S1_stdD_minsInv-len(S1_stdD_mins)
    print ("S1_stdD_mins", round(S1_stdD_mins.mean(),3), round(S1_stdD_mins.std(),3),len(S1_stdD_mins),S1_stdD_minsInv)

    S1_mean_minsInv = len(S1_mean_mins)
    S1_mean_mins = [ v for v in S1_mean_mins if v  >= 0]
    S1_mean_mins = np.array(S1_mean_mins)
    S1_mean_minsInv = S1_mean_minsInv-len(S1_mean_mins)
    print ("S1_mean_mins", round(S1_mean_mins.mean(),3), round(S1_mean_mins.std(),3),len(S1_mean_mins),S1_mean_minsInv)

    print (allmins.mean(),allmins.std(),len(allmins),invalidValues)
    file.close()
    

def getExecutionTimeStats():
    radius25  = "/home/milto/Documents/fieldData/statsNImgs/Radius25.txt" 
    radius50  = "/home/milto/Documents/fieldData/statsNImgs/Radius50.txt" 
    radius100 = "/home/milto/Documents/fieldData/statsNImgs/Radius100.txt" 
    print("******************\nRADIUS 25")
    getExecutionTimeStatsforFile(radius25)
    print("******************\nRADIUS 50")
    getExecutionTimeStatsforFile(radius50)
    print("******************\nRADIUS 100")
    getExecutionTimeStatsforFile(radius100)
